# 第4章: 形態素解析

In [6]:
# ファイルをコピー
!cp /content/drive/MyDrive/iyatomi/nlp100/neko.txt /content/

In [7]:
!pip install mecab-python3

In [12]:
!pip3 freeze | grep mecab

mecab-python3==1.0.3


In [13]:
!find / -iname mecabrc

find: ‘/proc/77/task/77/net’: Invalid argument
find: ‘/proc/77/net’: Invalid argument
find: ‘/proc/113/task/113/net’: Invalid argument
find: ‘/proc/113/net’: Invalid argument
find: ‘/proc/185/task/185/net’: Invalid argument
find: ‘/proc/185/net’: Invalid argument


## 30. 形態素解析結果の読み込み
形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [10]:
# マッピング型=辞書
import MeCab
tagger = MeCab.Tagger('-Ochasen')   #形態素解析器
with open('neko.txt') as f:
    text = f.read()
text_parsed = tagger.parse(text)
print(text_parsed)

----------------------------------------------------------



Failed initializing MeCab. Please see the README for possible solutions:

    https://github.com/SamuraiT/mecab-python3#common-issues

If you are still having trouble, please file an issue here, and include the
ERROR DETAILS below:

    https://github.com/SamuraiT/mecab-python3/issues

issueを英語で書く必要はありません。

------------------- ERROR DETAILS ------------------------
arguments: -Ochasen
error message: [ifs] no such file or directory: /usr/local/etc/mecabrc


RuntimeError: ignored

In [ ]:
# tagger.parse()だと文全体を解析処理した結果を返す
# 単語ごとの情報が欲しいのでtagger.parseToNode()を使用する
# ためしに最初の10文字を形態素解析
print(text[:11])
node = tagger.parseToNode(text[:11])
while node:
    # 表層形の表示
    print('surface', node.surface)
    # 品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音を表示
    print('feature', node.feature)
    ftr_list = node.feature.split(',')
    print('base', ftr_list[6])  # 原型(基本形？) 
    print('pos', ftr_list[0])   # 品詞
    print('pos1', ftr_list[1])  # 品詞細分類1

    node = node.next

In [ ]:
node = tagger.parseToNode(text)
node_list = []
while node:
    surface = node.surface  # 表層形
    if surface == '':
        node = node.next
        continue
    ftr_list = node.feature.split(',')
    base = ftr_list[6]  # 原型(基本形？) 
    pos = ftr_list[0]   # 品詞
    pos1 = ftr_list[1]  # 品詞細分類1
    # dict_node[(surface, base, pos, pos1)] = node # これは違う
    mor = {'surface':surface, 'base':base, 'pos':pos, 'pos1':pos1}
    node_list.append(mor)
    node = node.next
print(node_list[0])

## 31. 動詞
動詞の表層形をすべて抽出せよ．

In [ ]:
# これだと重複があるのでよくない
verb_list = [node['surface'] for node in node_list if node['pos']=='動詞']
print(verb_list)
# 重複なし
verb_set = {node['surface'] for node in node_list if node['pos']=='動詞'}
print(verb_set)
print(len(verb_set))

## 32. 動詞の基本形
動詞の基本形をすべて抽出せよ．

In [ ]:
# 重複なし
verb_base_set = {node['base'] for node in node_list if node['pos']=='動詞'}
print(verb_base_set)
print(len(verb_base_set))

## 33. 「AのB」
2つの名詞が「の」で連結されている名詞句を抽出せよ．

In [11]:
import re
import MeCab
tagger = MeCab.Tagger()   #形態素解析

noun_set = set()
for node in node_list:
    if node['pos'] != '名詞':
        continue
    noun = re.search(r'(.+)の(.+)', node['surface'])
    if noun:
        # print(noun.groups())
        head, tail = noun.groups()
        # print(head, tail)
    else:
        continue

    # print(tagger.parse(head).split('\t')[1].split(',')[0])
    if tagger.parse(head).split('\t')[1].split(',')[0] == '名詞' and tagger.parse(tail).split('\t')[1].split(',')[0] == '名詞':
        noun_set.add(node['surface'])
print(noun_set)
print(len(noun_set))

----------------------------------------------------------



Failed initializing MeCab. Please see the README for possible solutions:

    https://github.com/SamuraiT/mecab-python3#common-issues

If you are still having trouble, please file an issue here, and include the
ERROR DETAILS below:

    https://github.com/SamuraiT/mecab-python3/issues

issueを英語で書く必要はありません。

------------------- ERROR DETAILS ------------------------
arguments: 
error message: [ifs] no such file or directory: /usr/local/etc/mecabrc


RuntimeError: ignored

In [ ]:
con_noun_list = set()
# ↑だと名詞と判断された形態素をさらに分けることとなる
i = 1
while i < len(node_list)-2:
    if node_list[i-1]['pos'] == '名詞':
        node1 = node_list[i-1]['surface']
        node2 = node_list[i]['surface']
        node3 = node_list[i+1]['surface']
        # 名詞+の+名詞のならびならok
        if node2 == 'の' and node_list[i+1]['pos'] == '名詞':
            con_noun_list.add(node1+node2+node3)
            #i += 3 # 私の犬の名前　→　私の犬一つがヒットする('私'を指していたインデックスが次のループのときに(犬の後ろの)'の'を示すため)
            i += 1  # 私の犬の名前　→　私の犬、犬の名前の二つがヒットする
        else:
            i += 1
    else:
        i += 1
print(con_noun_list)
print(len(con_noun_list))

## 34. 名詞の連接
名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

In [ ]:
junc_noun_set = set()
junc_noun = []
for node in node_list:
    if node['pos'] == '名詞':
        junc_noun.append(node['surface'])
    else :
        if len(junc_noun) >= 2: # 2個以上連続して名詞があれば保存
            junc_noun_set.add(''.join(junc_noun))
        junc_noun = []

print(junc_noun_set)
print(len(junc_noun_set))

## 35. 単語の出現頻度
文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

In [ ]:
# 単語≠名詞？
# 記号以外が単語？
from collections import Counter
term_freqs = Counter()  # 各単語の頻度のCounterクラス(辞書のサブクラス)の辞書 term_freqs { '単語': 出現頻度' }
for node in node_list:
    if node['pos'] not in ['助詞', '助動詞', '記号', '連体詞'] and len(node['surface']) > 0:
        term_freqs[node['surface']] += 1
print(term_freqs)   # ほぼ辞書と同じ
print(term_freqs.most_common()) # 出現頻度の高い順で並べる

## 36. 頻度上位10語
出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

In [ ]:
terms = [term for term, freq in term_freqs.most_common()[:5]]
freqs = [freq for term, freq in term_freqs.most_common()[:5]]
print(terms)
print(freqs)

In [ ]:
# 日本語フォントつかいたい
! wget https://moji.or.jp/wp-content/ipafont/IPAexfont/ipaexg00401.zip

In [ ]:
!apt-get -y install fonts-ipafont-gothic

In [ ]:
# zip展開&フォント置場にフォント配置
! unzip ipaexg00401.zip
! cp -r ipaexg00401/ /usr/share/fonts/truetype

In [ ]:
# フォントを追加できた
!fc-list

In [ ]:
!ls -l /root/.cache/matplotlib/
!rm /root/.cache/matplotlib/fontlist-v310.json # 消すべきcache
!rm /root/.cache/matplotlib/fontList.json # 消すべきcache?

In [ ]:
!rm -r /root/.cache/matplotlib  # これ消して再起動？

In [ ]:
import matplotlib.pyplot as plt

plt.bar(terms, freqs)
# plt.rcParams['font.family'] = 'Humor-Sans'
plt.rcParams['font.family'] = 'IPAexGothic'
#plt.rcParams['font.family'] = 'IPAGothic'
# plt.rcParams['font.family'] = 'Gothic'
plt.show()

NameError: ignored

## 37. 「猫」と共起頻度の高い上位10語
「猫」とよく共起する（共起頻度が高い）10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

## 38. ヒストグラム
単語の出現頻度のヒストグラムを描け．ただし，横軸は出現頻度を表し，1から単語の出現頻度の最大値までの線形目盛とする．縦軸はx軸で示される出現頻度となった単語の異なり数（種類数）である．

## 39. Zipfの法則
単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．